<a href="https://colab.research.google.com/github/fabriciocarraro/Google-ADK_AI-Agent-for-social-media/blob/main/Google_ADK_AI_Agent_for_social_media.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [2]:
# Configurando a API Key do Google Gemini
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GEMINI_API_KEY')

In [10]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import warnings

warnings.filterwarnings("ignore")

In [4]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [15]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )
    # Prompt de entrada para o agente 1
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [14]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )
    # Prompt de entrada para o agente 2
    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [16]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    # Prompt de entrada para o agente 3
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [17]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    # Prompt de entrada para o agente 4
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [9]:
from datetime import date

data_de_hoje = date.today().strftime("%d/%m/%Y")

In [11]:
import textwrap # Para formatar melhor a saída de texto

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [18]:
print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# Obtendo o Tópico do Usuário
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Lógica da colaborações dos agentes
if not topico:
    print("\nVocê esqueceu de digitar o tópico!")
else:
    print(f"\nMaravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: IA
Maravilha! Vamos então criar o post sobre novidades em IA

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos de IA mais relevantes e atuais, focarei em notícias dos últimos meses e darei prioridade àqueles com maior cobertura e entusiasmo.
> 
> 
> Com base nas notícias mais recentes e relevantes sobre IA até maio de 2025, aqui estão alguns dos principais lançamentos e tendências:
> 
> 1.  **Modelos de IA mais avançados da OpenAI (o3 e o4-mini):** A OpenAI lançou modelos de raciocínio mais avançados, como o3 e o4-mini, capazes de raciocinar antes de responder, usar ferramentas automaticamente e até navegar na web quando necessário. O modelo o4-mini-high suporta até 1 milhão de tokens, ideal para analisar documentos longos. A OpenAI também lançou o Codex CLI, um agente de código leve e open-source.
> 2.  **Gemini 2.5 Pro e Flash do Google:** O Google apresentou o modelo Gemini Flash, que introduz o conceito de "orçamento de pensamento", equilibrando custo, velocidade e precisão, ideal para resumos, análises e interações complexas. Eles também revelaram o protocolo A2A (Agent2Agent), que permite a colaboração entre agentes de IA de diferentes plataformas.
> 3.  **Meta AI App:** A Meta lançou seu primeiro aplicativo de assistente com IA, competindo com o ChatGPT. O aplicativo permite que os usuários criem e compartilhem posts com IA, além de responder às suas necessidades. Ele funciona principalmente por comandos de voz, com interações adaptadas a cada usuário.
> 4.  **Aplicações de IA na Agrishow 2025:** A Agrishow 2025 destacou o uso crescente da IA no agronegócio, incluindo monitoramento e análise de dados, detecção e controle de pragas, previsão de clima e safras, otimização de irrigação e gestão da cadeia de suprimentos. A Solinftec apresentou a Refill Station, que permite que o Solix Ag Robotics opere de forma autônoma durante toda a safra.
> 5.  **Tendências de IA da Microsoft para 2025:** A Microsoft identificou seis tendências de IA para 2025, incluindo modelos mais capazes, agentes de IA, IA para acelerar descobertas científicas, IA mais pessoal e útil, e a importância da curadoria de dados de alta qualidade. Eles também destacaram o crescimento do uso de IA generativa nas empresas.
> 
> Além desses lançamentos específicos, algumas tendências gerais em IA incluem:
> 
> *   **Acessibilidade e custos de treinamento mais baixos:** A IA está se tornando mais acessível devido à redução dos custos de treinamento e otimização de arquiteturas.
> *   **Aprendizagem auto-supervisionada e IA explicável:** Essas tendências estão tornando os sistemas de IA mais robustos, interpretáveis e escaláveis.
> *   **Integração da IA em diversas aplicações cotidianas:** A IA está sendo integrada em diversas aplicações, desde assistentes virtuais até ferramentas de criação de conteúdo visual.
> *   **Adoção de agentes inteligentes:** Espera-se que 25% das empresas adotem agentes inteligentes em 2025.
> 
> É importante notar que o campo da IA está em constante evolução, e novas ferramentas e modelos estão surgindo a todo momento.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Com base nos lançamentos e tendências de IA que você listou, aqui está uma análise para planejar um post de mídia social impactante:
> 
> **Análise dos Lançamentos:**
> 
> 1.  **OpenAI (o3 e o4-mini):**
>     *   **Relevância:** Alta, devido ao avanço das capacidades de raciocínio e automatização, além do suporte para documentos longos.
>     *   **Potenciais Ângulos:**
>         *   "A OpenAI eleva a IA a outro nível: modelos que pensam por você e automatizam tarefas complexas."
>         *   "Como o o4-mini pode transformar a análise de documentos longos e a tomada de decisões?"
>         *   "OpenAI Codex CLI: O que é e como ele pode revolucionar o desenvolvimento de código open-source?"
>     *   **Possíveis buscas para complementar:** "Casos de uso OpenAI o4-mini", "OpenAI Codex CLI tutorial".
> 2.  **Google (Gemini 2.5 Pro e Flash):**
>     *   **Relevância:** Alta, especialmente o Gemini Flash com o conceito de "orçamento de pensamento" e o protocolo A2A.
>     *   **Potenciais Ângulos:**
>         *   "Google Gemini Flash: A IA que equilibra custo e precisão. O futuro dos resumos e análises rápidas?"
>         *   "A2A (Agent2Agent): Como a colaboração entre IAs de diferentes plataformas vai mudar o jogo?"
>     *   **Possíveis buscas para complementar:** "Comparativo Gemini Flash vs outros modelos", "Exemplos de uso A2A Google".
> 3.  **Meta AI App:**
>     *   **Relevância:** Média, devido à entrada da Meta no mercado de assistentes de IA e a integração com a criação de conteúdo.
>     *   **Potenciais Ângulos:**
>         *   "Meta AI App chegou para competir com o ChatGPT! Crie posts com IA e tenha um assistente pessoal no seu bolso."
>         *   "Comandos de voz e IA adaptada: o que esperar do aplicativo de assistente da Meta?"
>     *   **Possíveis buscas para complementar:** "Reviews Meta AI App", "Tutorial Meta AI App".
> 4.  **Agrishow 2025:**
>     *   **Relevância:** Média, mas com um nicho específico e crescente.
>     *   **Potenciais Ângulos:**
>         *   "IA no Agro: como a tecnologia está transformando a produção de alimentos na Agrishow 2025."
>         *   "Solix Ag Robotics: o futuro da agricultura autônoma já é realidade?"
>     *   **Possíveis buscas para complementar:** "Resultados Agrishow 2025", "Impacto da IA na agricultura".
> 5.  **Microsoft (Tendências de IA 2025):**
>     *   **Relevância:** Alta, pois aponta para o futuro da IA.
>     *   **Potenciais Ângulos:**
>         *   "As 6 tendências de IA da Microsoft para 2025 que você precisa conhecer."
>         *   "IA para ciência, agentes inteligentes e curadoria de dados: o que esperar do futuro da IA segundo a Microsoft?"
>     *   **Possíveis buscas para complementar:** "Relatório completo tendências IA Microsoft 2025", "Aplicações práticas tendências IA Microsoft".
> 
> **Escolha do Tema Mais Relevante e Plano de Post:**
> 
> Com base na análise, o tema mais relevante é a combinação dos **Modelos de IA da OpenAI (o3 e o4-mini) e do Google (Gemini 2.5 Pro e Flash)**. Eles representam avanços significativos e têm aplicações amplas.
> 
> **Plano de Post:**
> 
> *   **Título:** "OpenAI vs Google: A Batalha da IA Acelera com Modelos Inovadores!"
> *   **Introdução (20% do post):**
>     *   Contextualizar a rápida evolução da IA.
>     *   Mencionar o lançamento dos modelos o3/o4-mini (OpenAI) e Gemini Flash (Google).
>     *   Instigar o leitor a descobrir como esses modelos estão mudando o jogo.
> *   **OpenAI (30% do post):**
>     *   Explicar as capacidades de raciocínio e automatização dos modelos o3 e o4-mini.
>     *   Destacar o suporte para análise de documentos longos e o potencial do Codex CLI.
>     *   Usar exemplos práticos de como esses modelos podem ser utilizados.
> *   **Google (30% do post):**
>     *   Apresentar o conceito de "orçamento de pensamento" do Gemini Flash e como ele equilibra custo e precisão.
>     *   Explicar o protocolo A2A e seu potencial para colaboração entre IAs.
>     *   Mostrar como o Gemini Flash pode ser usado em resumos, análises e interações complexas.
> *   **Comparativo e Conclusão (20% do post):**
>     *   Comparar os pontos fortes de cada modelo e suas aplicações.
>     *   Discutir como esses avanços afetam o futuro da IA e o mercado.
>     *   Incentivar o debate nos comentários: "Qual desses modelos você acha mais promissor? Deixe sua opinião!"
> *   **Call to Action:**
>     *   "Compartilhe este post para que mais pessoas fiquem por dentro das novidades da IA!"
>     *   "Siga nossa página para mais conteúdo sobre tecnologia e inovação."
> 
> Este plano fornece uma estrutura para um post envolvente e informativo, capaz de atrair a atenção do público e gerar discussões relevantes.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> Ok, aqui está um rascunho de post para o Instagram da Alura, seguindo o plano e buscando engajamento:
> 
> ---
> 
> **[IMAGEM/VÍDEO RECOMENDADO: Uma arte gráfica dinâmica comparando logos da OpenAI e Google com ícones representando IA, raciocínio, velocidade e colaboração.]**
> 
> **Texto do Post:**
> 
> 🤖 OpenAI vs Google: A Batalha das IAs está PEGANDO FOGO! 🔥
> 
> O universo da IA não para! 🤯 Você piscou e a OpenAI e o Google lançaram modelos que prometem revolucionar TUDO. Estamos falando do o3/o4-mini e do Gemini Flash! 🚀
> 
> Mas o que isso significa NA PRÁTICA? 🤔
> 
> ✨ **Do lado da OpenAI:** Os modelos o3 e o4-mini chegam turbinados! Pense em IAs com mais capacidade de raciocínio, que automatizam tarefas complexas e até analisam documentos gigantescos (adeus, leitura de contratos intermináveis? 🙏). E para devs, o Codex CLI promete ser um aliado poderoso na hora de codar!
> 
> ⚡ **Na resposta do Google:** O Gemini Flash traz o conceito inovador de "orçamento de pensamento" 🧠. Ele equilibra custo e precisão, sendo perfeito para resumos rápidos e análises eficientes. Além disso, o protocolo A2A (Agent2Agent) abre portas para IAs de diferentes plataformas COLABORAREM entre si. Já imaginou o potencial disso? 🤝
> 
> 🥊 **E aí, quem leva?** Enquanto a OpenAI aposta em poder bruto e automação, o Google foca na eficiência e colaboração. O resultado? Ferramentas cada vez mais incríveis chegando até você!
> 
> O futuro da tecnologia está sendo moldado AGORA, e esses avanços impactam desde o desenvolvimento de software até a forma como consumimos informação.
> 
> 🤔 Na sua opinião, qual desses avanços vai impactar mais a sua vida ou trabalho? Conta pra gente nos comentários! 👇
> 
> Curtiu ficar por dentro das novidades? Compartilhe este post com a galera que AMA tecnologia! ✨ E siga a @aluraonline para não perder nada sobre IA e inovação! 😉
> 
> #InteligenciaArtificial #Tecnologia #Inovacao #Google #OpenAI


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho tem uma ótima base, com um tom engajador e formato adequado para o Instagram e o público jovem! No entanto, há alguns pontos de precisão técnica que precisam de ajuste para garantir a correção da informação:
> 
> 1.  **Nomes dos Modelos OpenAI:** A referência a "o3/o4-mini" está incorreta. O modelo principal anunciado recentemente pela OpenAI foi o **GPT-4o** (o "o" significa "omni"). Ele é mais rápido, mais barato e integra capacidades de texto, áudio e visão. Não existe um "o3" ou "o4-mini" nesses termos.
>     *   **Sugestão:** Substituir "o3/o4-mini" por "**GPT-4o**" ou, de forma mais genérica, "novos modelos como o GPT-4o".
> 
> 2.  **Referência ao Codex CLI:** Embora o GPT-4o melhore as capacidades de codificação, mencionar especificamente o "Codex CLI" pode não ser a forma mais precisa de descrever o anúncio recente, que teve mais foco nas capacidades multimodais e na disponibilidade.
>     *   **Sugestão:** Trocar "E para devs, o Codex CLI promete ser um aliado poderoso na hora de codar!" por algo como "E para devs, as **melhorias na API e a velocidade** prometem turbinar o desenvolvimento!" ou focar nas capacidades gerais de auxílio à codificação aprimoradas.
> 
> 3.  **Protocolo A2A (Agent2Agent) do Google:** A menção ao "protocolo A2A (Agent2Agent)" pode ser uma simplificação ou interpretação. O Google apresentou o **Project Astra** e falou sobre o futuro dos agentes de IA, incluindo a capacidade de colaborarem, mas "A2A" não foi um termo oficial amplamente divulgado como um protocolo específico neste contexto recente (Google I/O).
>     *   **Sugestão:** Reformular para algo como: "Além disso, a visão do Google com o **Project Astra** e o avanço dos **agentes de IA** abre portas para IAs colaborarem entre si de formas inovadoras. Já imaginou o potencial disso? 🤝" ou "Além disso, o Google aposta em **agentes de IA mais inteligentes e colaborativos**, capazes de realizar tarefas complexas no seu lugar. Já imaginou o potencial disso? 🤝".
> 
> **Resumo das Sugestões:**
> 
> *   Corrigir os nomes dos modelos da OpenAI para **GPT-4o**.
> *   Ajustar a menção ao benefício para desenvolvedores da OpenAI para algo mais geral ou focado nas melhorias de API/velocidade anunciadas.
> *   Reformular a parte sobre a colaboração de IAs do Google, talvez mencionando o **Project Astra** ou o conceito de **agentes de IA colaborativos** em vez do "protocolo A2A".
> 
> Com essas correções, o post ficará mais preciso tecnicamente, mantendo o tom e o engajamento excelentes!


--------------------------------------------------------------
